# Python Date and Time

## Intro

### Python has 5 classes:

    - Date: Year / Month / Day
    - Time: Hour/ Minutes / Seconds / Microseconds 
    - Datetime: = Date + Time
    - Timedelta: datetime.timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
        -> returns a date. Commonly used for adding/substracting from an initial date, creating a new date, a delta away.
    - tzinfo: is a 'abstract' class with the timezone info, abstract meaning that it can not be created directly
   
## Concept of "Aware" and "Naive" objects

    - Naive date = date with no context of timezone
        -> eg: datatime.datetime(2020,12,25,20,30,15)
            -> It is like a watch indicating 20h30, but are you in New York, Sydney, Brussels or ...?
            -> when printed there will be NO reference to UTC like +01:00
           
    - Aware date = Native date +  a timezone
    
Python doesn not have build-in timezone support. 

**You have to use:**

    - pytz 
    or
    - dateutils



What challenges do we have?

    - a time should be referenced to a timezone
    - we have to be able to cope with DST - Daylight Saving Time. When DST is applied in a timezone, we can be confronted with two times the same time when the clock is reset 1 hour. That night we haven 02h:00 and 1 hour later again 02h:00

In [60]:
import datetime
unaware = datetime.datetime(2020,12,26,16,4,59)
print(unaware)

2020-12-26 16:04:59


In [61]:
# now() creates an unaware date ! so no timezone info or context
unaware = datetime.datetime.now()
print(unaware)

2020-12-27 15:28:27.948268


In [62]:
# if we only want seconds and no milli or microseconds
only_secs = unaware.replace(microsecond =0)
print(only_secs)
# timestamp converts to amount of secs from 1/1/1970
print(only_secs.timestamp())

2020-12-27 15:28:27
1609079307.0


## Making 'naive' time, time 'Aware'

In order to do so we need a 'timezone object', so we can **link** our "unaware' time to a defined timezone and create an **'Aware'** time.

We have 2 possible solutions (libraries) to create a timezone object:

    - pytz: 
        -> step 1: create a timezone object
        -> step 2: link the time to the timezone via 'localize' or astimezone()
    - dateutil:
        -> step 1: create a timezone object
        -> step 2: create your time with tzinfo = 'yourTimeZone' or use astimezone(

### How to create a timezone object with pytz

In [63]:
import pytz
import datetime

brussels = pytz.timezone('Europe/Brussels')

### How to create a timezone object with dateutil

In [64]:
from dateutil import tz
import datetime

brugge =tz.gettz('Europe/Brussels')

### How to create a 'Aware' time with pytz

**Important !:** We can NOT use the pytz created timezone in the datetime object -> tzinfo = myTimeZone !!!

eg: datetime.datetime(2020,12,25,20,30,15, tzinfo = brussels)

In [65]:
import pytz
brussels=pytz.timezone('Europe/Brussels')

unaware = datetime.datetime(2020,12,26,16,4,59)

# Method 1: with localize
aware10 = brussels.localize(datetime.datetime(2020,12,26,16,4,59))
print(aware10)

# Method 2: with astimezone
aware11 = unaware.astimezone(brussels)
print(aware11)

2020-12-26 16:04:59+01:00
2020-12-26 16:04:59+01:00


### How to create a 'Aware' time with dateutil

In [66]:
from dateutil import tz
import datetime

brugge =tz.gettz('Europe/Brussels')

# Method 1: directly when creating the time (something that can NOT be done with puytz !!)
aware20 = datetime.datetime(2020,12,26,16,4,59,tzinfo=brugge)
aware21 = datetime.datetime.utcnow().astimezone(tz=brugge)

unaware20= datetime.datetime.now()

# Method 2: with astimezone
aware22= unaware20.astimezone(tz=brugge)
aware23= datetime.datetime.now().astimezone(tz=tz.UTC)

print(aware20)
print(aware21)
print(unaware10)
print(aware22)
print(aware23)

2020-12-26 16:04:59+01:00
2020-12-27 14:28:53.100898+01:00
2020-12-27 15:04:35.598543
2020-12-27 15:28:53.100971+01:00
2020-12-27 14:28:53.101049+00:00


## DST - Daylight Saving Time

This is usefull to indicate the difference between 02h00 and 02h00, one hour later when DST change takes place

In [72]:
my_summerdate = datetime.datetime(2020,7,1,8,59,59,tzinfo=brugge)

# dst() indicates of Daylight Saving Time is active or not
print(my_summerdate.dst())

# my_first_02H00 at DST change
unaware = datetime.datetime(2020,10,25,2,0,0)

my_first_02h00 = brussels.localize(unaware,is_dst=True)
my_second_02h00 = brussels.localize(unaware,is_dst=False)
print(my_first_02h00)
print(my_first_02h00.timestamp())
print(my_second_02h00)
print(my_second_02h00.timestamp())

1:00:00
2020-10-25 02:00:00+02:00
1603584000.0
2020-10-25 02:00:00+01:00
1603587600.0


## Iso 8601 = defacto standard

    - format: YYYY-MM-DDThh:mm:ss+UTC offset or 'Z = Zulu time = +00:00'
        - example: 2020-12-26T16:04:59+01:00
    - format: seconds since 1/1/1970
        -> ideal as timestamp in databases
        -> gives microsecs precision
            example: 2020-12-26 17:18:13.894596 = 1608999493.894596 
                -> 1608999493 secs
                -> 894 millisecs
                -> 596 microsecs

In [32]:
my_iso_datetime = aware.isoformat()
print(my_iso_datetime)

2020-12-26T16:04:59+01:00


In [33]:
my_iso_datetime_in_utc = aware.astimezone(pytz.UTC).isoformat()
print(my_iso_datetime_in_utc)

2020-12-26T15:04:59+00:00


In [74]:
my_timestamp = aware.timestamp()
print(aware)
print(my_timestamp)
print(datetime.datetime.fromtimestamp(my_timestamp,tz=pytz.utc).isoformat()+'Z')

2020-12-26 16:04:59+01:00
1608995099.0
2020-12-26T15:04:59+00:00Z


## Timedelta example

datetime.timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)

In [40]:
today =datetime.datetime.now()
tomorrow_sametime = today + datetime.timedelta(days=1)
print(today)
print(tomorrow_sametime)

2020-12-26 17:18:13.894596
2020-12-27 17:18:13.894596


## Example:

In [73]:
# we created 2 timezones: brussels and utc = +00:00 = Z (Zulu)
brussels = tz.gettz('Europe/Brussels')
my_utc_timezone = tz.UTC

# we create a datetime -> now() is 'naive -> with no context of knowledge of timezone'
my_time = datetime.datetime.now()

# we create 2 timezone 'Aware' times: both are NOT the same "occurence". eg same 'clock' but exist in a different timezone
my_local_time = my_time.astimezone(tz=brussels)
my_utc_time = my_time.astimezone(tz=my_utc_timezone)

print(my_local_time)
print(my_utc_time)

# My clock in Brussels but seen and referenced from the "zulu" UTC timezone (+00:00) aka GMT
print(my_local_time.astimezone(tz.UTC))
print(my_local_time.astimezone(tz.UTC).isoformat()[:-6]+'Z')


2020-12-27 15:46:41.912484+01:00
2020-12-27 14:46:41.912484+00:00
2020-12-27 14:46:41.912484+00:00
2020-12-27T14:46:41.912484Z


## List all available timezones

In [49]:
for tz in pytz.all_timezones:
    print(tz)

Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara
Africa/Asmera
Africa/Bamako
Africa/Bangui
Africa/Banjul
Africa/Bissau
Africa/Blantyre
Africa/Brazzaville
Africa/Bujumbura
Africa/Cairo
Africa/Casablanca
Africa/Ceuta
Africa/Conakry
Africa/Dakar
Africa/Dar_es_Salaam
Africa/Djibouti
Africa/Douala
Africa/El_Aaiun
Africa/Freetown
Africa/Gaborone
Africa/Harare
Africa/Johannesburg
Africa/Juba
Africa/Kampala
Africa/Khartoum
Africa/Kigali
Africa/Kinshasa
Africa/Lagos
Africa/Libreville
Africa/Lome
Africa/Luanda
Africa/Lubumbashi
Africa/Lusaka
Africa/Malabo
Africa/Maputo
Africa/Maseru
Africa/Mbabane
Africa/Mogadishu
Africa/Monrovia
Africa/Nairobi
Africa/Ndjamena
Africa/Niamey
Africa/Nouakchott
Africa/Ouagadougou
Africa/Porto-Novo
Africa/Sao_Tome
Africa/Timbuktu
Africa/Tripoli
Africa/Tunis
Africa/Windhoek
America/Adak
America/Anchorage
America/Anguilla
America/Antigua
America/Araguaina
America/Argentina/Buenos_Aires
America/Argentina/Catamarca
America/Argentina/ComodRivad

# Working with the time library

In [33]:
import time
ts_in_seconds = time.time()
ts_as_tuple = time.gmtime()
ts_as_string = time.strftime("%Y-%m-%dT%H:%M:%S", ts_as_tuple)
ts_as_iso_string = time.strftime("%Y-%m-%dT%H:%M:%S", ts_as_tuple)+ "+00:00Z"

In [29]:
print(ts_in_seconds)
print(ts_as_tuple)
print(ts_as_string)
print(ts_as_iso_string)

1609598177.087612
time.struct_time(tm_year=2021, tm_mon=1, tm_mday=2, tm_hour=14, tm_min=36, tm_sec=17, tm_wday=5, tm_yday=2, tm_isdst=0)
2021-01-02T14:36:17
2021-01-02T14:36:17+00:00Z


In [37]:
print(time.ctime(ts_in_seconds))

Sun Jan  3 15:02:53 2021


In [9]:
iso_time = time.strftime("%Y-%m-%dT%H:%M:%S", tuple_time)
print(iso_time)

2021-01-02T10:19:06


In [30]:
# When you need the Iso8601 time format
import time
import datetime
import pytz
my_timestamp = time.time()
print(my_timestamp)
print(datetime.datetime.fromtimestamp(my_timestamp,tz=pytz.utc).isoformat()+'Z')

1609608129.8201559
2021-01-02T17:22:09.820156+00:00Z
